In [57]:
"""
Before running this cell:
1) $ cd ~/Documents/NCS/cortexcontrol
2) $ sudo ./cortexcontrol
3) $ exec(open("./start_rpyc.py").read())
"""

import rpyc
c = rpyc.classic.connect("localhost", 1300)
RPYC_TIMEOUT = 300 #defines a higher timeout
c._config["sync_request_timeout"] = RPYC_TIMEOUT  # Set timeout to higher level

# cortexcontrol modules are accessed through the connection.modules attribute
print(c.modules.CtxDynapse) # prints module address

<module 'CtxDynapse' (built-in)>


In [58]:
"""
Created on Thu Sep 26 22:11:04 2019

@author: dzenn
"""
import numpy as np
from random import sample
from time import sleep, time
import operator

class SBSController():
    """
        Spike By Spike project Dynapse chip controller class
        
    """
    
    def __init__(self, start_neuron, chip_id, c, core_id, debug = False, base_isi=90):
        
        self.start_neuron = start_neuron
        self.num_neurons = 20
        self.chip_id = chip_id
        self.core_id = core_id
        self.debug = debug
        self.c = c
        self.base_isi = base_isi
        
        try:
            print("rpyc ok")
            self.model = c.modules.CtxDynapse.model
            print("model ok")
            self.vModel = self.c.modules.CtxDynapse.VirtualModel()
            print("vModel ok")
            self.SynTypes = c.modules.CtxDynapse.DynapseCamType
            print("SynTypes ok")
            self.dynapse = c.modules.CtxDynapse.dynapse
            print("dynapse ok")
            self.connector = c.modules.NeuronNeuronConnector.DynapseConnector()
            print("connector ok")
            self.fpga_spike_event = c.modules.CtxDynapse.FpgaSpikeEvent
            print("Spike event ok")
        except AttributeError:
            print("Init failed: RPyC connection not active!")
            return
        
                
        self.spikegen = self.model.get_fpga_modules()[1]
        
        self.neurons = self.model.get_shadow_state_neurons()
        self.v_neurons = self.vModel.get_neurons()
        self.bias_group = self.model.get_bias_groups()[chip_id*4 + core_id]
        
        ADDR_NUM_BITS = 15
        ISI_NUM_BITS = 16
        self.max_fpga_len = 2**ADDR_NUM_BITS-1
        self.max_isi = 2**ISI_NUM_BITS-1 
        
        self.population = [n for n in self.neurons if n.get_chip_id()==self.chip_id
             and n.get_core_id()==self.core_id
             and n.get_neuron_id() >= self.start_neuron
             and n.get_neuron_id() < self.start_neuron + self.num_neurons]
        self.population_ids = [n.get_chip_id()*1024 + n.get_core_id()*256 + n.get_neuron_id() for n in self.population]
        
    @classmethod
    def from_default(self):
        return SBSController(start_neuron=1, chip_id=1, c=c, core_id=1, debug=False)
        
    def run_single_trial(self):
        
        pass
    
    def get_fpga_events(self, fpga_isi, fpga_nrn_ids):
        """ This function takes a list of events and neuron ids and returns an 
        object of FpgaSpikeEvent class.
        Args:
            fpga_isi     (list): list of isi 
            fpga_nrn_ids (list): list of neuron ids
        Returns:
            fpga_event   (FpgaSpikeEvent): ctxctl object
        """
        fpga_events = []
        for idx_isi, isi in enumerate(fpga_isi):
            fpga_event = self.fpga_spike_event()
            fpga_event.core_mask = 15
            fpga_event.target_chip = self.chip_id
            fpga_event.neuron_id = fpga_nrn_ids[idx_isi]
            fpga_event.isi = isi
            fpga_events.append(fpga_event)
        
        return fpga_events
    
    # Spike times in millis
    def spikes_to_isi(self, spike_times, neurons_id, use_microseconds=False):
        Signal_isi = []
        for i in range(len(spike_times)):
            if i == 0 :
                Signal_isi.append(spike_times[0])
            else:
                Signal_isi.append(spike_times[i] - spike_times[i-1])
        Signal_isi = np.asarray(Signal_isi)
        if(use_microseconds):
            Signal_isi = Signal_isi * 1e3
        else: # Already in microseconds
            Signal_isi = Signal_isi

        # Avoid using neuron zero (because all neurons are connected to it)
        if(0 in neurons_id):
            neurons_id = neurons_id + 1 
        return (Signal_isi, neurons_id)
    
    def load_spikeGen(self, fpga_events, isibase, repeat_mode=False):
        """ This loads an FpgaSpikeEvent in the Spike Generator.
        Args:
            fpga_events (FpgaSpikeEvent):
            isibase     (isibase):
        """ 
        self.spikegen.set_variable_isi(True)
        self.spikegen.preload_stimulus(fpga_events)
        self.spikegen.set_isi_multiplier(isibase)
        self.spikegen.set_repeat_mode(repeat_mode)
        
    def load_resources(self):
        """
            Loading all resource files: input weights and preloaded spiketrains
        """
        
        self.input_weights = np.load("Resources/F.dat", allow_pickle=True)
        self.spikes_0_up = np.load("Resources/x0_up.dat", allow_pickle=True)
        self.spikes_0_down = np.load("Resources/x0_down.dat", allow_pickle=True)
        self.spikes_1_up = np.load("Resources/x1_up.dat", allow_pickle=True)
        self.spikes_1_down = np.load("Resources/x1_down.dat", allow_pickle=True)
        
        fpga_evts = self.compile_preloaded_stimulus(dummy_neuron_id = 255)
        
        # Convert to ISI
        (signal_isi, neuron_ids) = self.spikes_to_isi(spike_times=fpga_evts[:,1], neurons_id=fpga_evts[:,0], use_microseconds=False)

        # Get the FPGA events
        fpga_events = self.get_fpga_events(signal_isi, neuron_ids)
        
        # Load spike gen
        self.load_spikeGen(fpga_events, self.base_isi, repeat_mode=False)
        
        
    
    def compile_preloaded_stimulus(self, dummy_neuron_id = 255):
        
        output_events = []
        for timestamp in self.spikes_0_up:
            output_events.append([1, int(timestamp*1000)])
        for timestamp in self.spikes_0_down:
            output_events.append([2, int(timestamp*1000)])
        for timestamp in self.spikes_1_up:
            output_events.append([3, int(timestamp*1000)])
        for timestamp in self.spikes_1_down:
            output_events.append([4, int(timestamp*1000)])
          
        output_events.sort(key=operator.itemgetter(1))
        output_events = np.insert(output_events, 0, [dummy_neuron_id,0], axis = 0)
        
        tmp_id = 1
        while tmp_id < len(output_events):
            if output_events[tmp_id,1] - output_events[tmp_id-1,1] > self.max_isi:
                output_events = np.insert(output_events, tmp_id, [dummy_neuron_id,output_events[tmp_id-1,1]+self.max_isi-1], axis = 0)
            tmp_id += 1
            
            
        return output_events
        
    def plot_raster(self):
        pass


In [59]:
sbs = SBSController.from_default()
sbs.load_resources()


rpyc ok
model ok
vModel ok
SynTypes ok
dynapse ok
connector ok
Spike event ok
